In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [3]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol='BTCUSDT',
    timeframe='1h',
    candles_to_dl=500,
)

In [4]:
candles_to_df(candles)

,timestamp,open,high,low,close,volume
datetime,,,,,,
2023-11-16 20:00:00,1700164800000,35929.11,36067.20,35671.57,35953.92,12.37658
2023-11-16 21:00:00,1700168400000,35954.25,36050.00,35833.42,35962.08,7.92226
2023-11-16 22:00:00,1700172000000,35954.00,36311.40,35950.00,36272.86,17.84238
2023-11-16 23:00:00,1700175600000,36272.85,36272.85,36068.00,36209.38,8.10998
2023-11-17 00:00:00,1700179200000,36197.88,36373.10,36129.32,36352.40,10.65019
...,...,...,...,...,...,...
2023-12-07 11:00:00,1701946800000,43120.06,43397.80,43120.06,43183.09,5.02346
2023-12-07 12:00:00,1701950400000,43195.36,43404.51,42961.77,43404.51,19.53573
2023-12-07 13:00:00,1701954000000,43392.82,43505.80,43195.23,43296.07,10.55007


In [7]:
from quantfreedom.enums import CandleBodyType


timestamp = candles[:, CandleBodyType.Timestamp]
open = candles[:, CandleBodyType.Open]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]

In [8]:
source = close
length = 50

In [16]:
sma = np.full_like(source, np.nan)
len_minus_one = length - 1
starting_index = source[np.isnan(source)].size + len_minus_one

In [17]:
for i in range(starting_index, source.size):
    sma[i] = source[i - len_minus_one : i + 1].mean()

In [18]:
sma

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 36393.78,
       36404.60, 36416.73, 36420.77, 36426.41, 36428.72, 36429.29,
       36430.56, 36432.18, 36435.85, 36441.07, 36446.16, 36454.19,
       36456.74, 36459.21, 36461.70, 36463.90, 36462.81, 36467.49,
       36478.56, 36489.32, 36506.02, 36514.84, 36523.54, 36536.73,
       36546.10, 36564.86, 36579.48, 36593.45, 36610.61, 36624.44,
       36640.07, 36654.55, 36669.61, 36688.09, 36705.72, 36721.33,
       36738.11, 36753.14, 36766.65, 36780.18, 36800.33, 36819.52,
       36834.70, 36852.94, 36869.85, 36888.45, 36904.87, 36919.57,
       36936.90, 36953.75, 36972.00, 36991.82, 37011.33, 37030.47,
       37049.06, 37065.12, 37081.61, 37096.19, 37112.01, 37125.87,
       37140.47, 37151.54, 37164.60, 37175.83, 

In [19]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=pd.to_datetime(timestamp, unit="ms"),
            open=open,
            high=high,
            low=low,
            close=close,
            name="candles",
        ),
        go.Scatter(
            x=pd.to_datetime(timestamp, unit="ms"),
            y=sma,
            name="SMA",
        ),
    ]
)
fig.update_layout(height=800, xaxis_rangeslider_visible=False)
fig.show()